<a href="https://colab.research.google.com/github/moveformyfuture/KRX-Big-Data-Competition/blob/main/proto_regulation_analysis_nlp_ver3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# KRX 금융 데이터 활용한 빅데이터 경진대회(Dacon'22)

> 작성일: '22.7/20  
> 팀명: 흔들리는 주가 속에서, 네 규제향이 느껴진거야   
> 아이디어명: KoBERT를 활용한 최신규제에 대한 **종목 영향도** 및 **핵심주제**, **피규제 종목 알림** 분석 서비스 개발  
> 작성자: 박창준(팀리더), 김미선, 이성욱, 설민우  

In [1]:
from google.colab import drive
drive.mount('/content/gdrive/')

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


In [2]:
!pip install selenium
!pip install webdriver_manager
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached urllib3-1.26.10-py2.py3-none-any.whl (139 kB)
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.25.11
    Uninstalling urllib3-1.25.11:
      Successfully uninstalled urllib3-1.25.11
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
requests 2.23.0 requires urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you have urllib3 1.26.10 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached urllib3-1.25.11-py2.py3-none-any.whl (127 kB)
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.26.10
    Uninstalling urllib3-1.26.10:
      Successfully

In [3]:
!pip install KoNLPy
!pip install pynori

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
!pip install olefile #한글문서(hwp) 파싱을 위해 필요한 패키지
import os
os.chdir("/content/gdrive/Shareddrives/Dacon_krx/code")
os.listdir()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


['proto_regulation_analysis_nlp_ver3.ipynb',
 'data_collect',
 'data_modeling',
 'data_preprocessing',
 'file',
 'Mecab-ko-for-Google-Colab',
 'proto_regulation_analysis_nlp.ipynb',
 'proto_regulation_analysis_nlp_ver2.ipynb']

In [5]:
'''
# from data_collect.parse_HWPfile import FinanaceRegulation_HWP_Parser
'''

import enum
import olefile
import pandas as pd
import os
import zlib
import struct
import re


#데이터프레임 중간 행,열 생략없이 모두 출력하는 설정
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', -1)

class FinanaceRegulation_HWP_Parser:
    def __init__(self, fname, directory):
        self.root = os.path.join(directory, fname)
        self.fname = fname

        f = olefile.OleFileIO(self.root)
        dirs = f.listdir()
        # HWP 파일 검증
        if ["FileHeader"] not in dirs or \
            ["\x05HwpSummaryInformation"] not in dirs:
            raise Exception("Not Valid HWP.")
        
        # 문서 포맷 압축 여부 확인
        header = f.openstream("FileHeader")
        header_data = header.read()
        is_compressed = (header_data[36] & 1) == 1

        # Body Sections 불러오기
        nums = []
        for d in dirs:
            if d[0] == "BodyText":
                nums.append(int(d[1][len("Section"):]))
        sections = ["BodyText/Section"+str(x) for x in sorted(nums)]

        # 전체 text 추출
        text = ""
        for seciton in sections:
            bodytext = f.openstream(seciton)
            data = bodytext.read()
            if is_compressed:
                unpacked_data = zlib.decompress(data,-15)
            else:
                unpacked_data = data
        
            # 각 Section 내 text 추출
            section_text = ""
            i = 0
            size = len(unpacked_data)
            while i < size:
                header = struct.unpack_from("<I", unpacked_data, i)[0]
                rec_type = header & 0x3ff
                rec_len = (header >> 20) & 0xfff

                if rec_type in [67]:
                    rec_data = unpacked_data[i+4:i+4+rec_len]
                    section_text += rec_data.decode('utf-16')
                    section_text += "\n"

                i += 4 + rec_len

            text += section_text
            text += "\n"

        self.raw_text = text

    def parse_from_text(self):

        text_line_token =re.sub(r"[^a-zA-Z0-9가-힣.\-\r\n\s]", '', self.raw_text)
        for idx, line in enumerate(text_line_token.splitlines()):
            match_result = re.match("\d{1,2}", line)
            if match_result:
                if (line == "1.규제사무명"):
                    start_idx1 = idx
                elif (line == "2.규제조문"):
                    start_idx2 = idx
                elif (line == "3.위임법령"):
                    start_idx3 = idx
                elif (line == "4.유형"):
                    start_idx4 = idx
                elif (line == "5.입법예고"):
                    start_idx5 = idx
                elif (line == "6.추진배경"):
                    start_idx6 = idx
                elif (line == "7.규제내용"):
                    start_idx7 = idx
                elif (line == "8.피규제집단      및"):
                    start_idx8 = idx
                elif (line == "9.도입목표 및" or line =="9.규제목표"):
                    start_idx9 = idx
                elif (line == "10.비용편익분석" or line == "10.영향평가 여부"):
                    start_idx10 = idx
                elif (line == "11.영향평가 여부" or line == "11. 비용편익"):
                    start_idx11 = idx

        contents_dict = dict()
        try: 
            contents_dict["1.규제사무명"] =  ' '.join((text_line_token.splitlines())[start_idx1+1:start_idx2])  
            contents_dict["2.규제조문"] =  ' '.join((text_line_token.splitlines())[start_idx2+1:start_idx3])
            contents_dict["3.위임법령"] =  ' '.join((text_line_token.splitlines())[start_idx3+1:start_idx4])
            contents_dict["4.유형"] =  ' '.join((text_line_token.splitlines())[start_idx4+1:start_idx5])
            contents_dict["5.입법예고"] =  ' '.join((text_line_token.splitlines())[start_idx5+1:start_idx6])
            contents_dict["6.추진배경및정부개입필요성"] =  ' '.join((text_line_token.splitlines())[start_idx6+2:start_idx7])
            contents_dict["7.규제내용"] =  ' '.join((text_line_token.splitlines())[start_idx7+1:start_idx8])
            contents_dict["8.피규제집단및이해관계자"] =  ' '.join((text_line_token.splitlines())[start_idx8+2:start_idx9])
            contents_dict["9.도입목표및기대효과"] =  ' '.join((text_line_token.splitlines())[start_idx9+2:start_idx10])
            contents_dict["10.비용편익분석"] =  ' '.join((text_line_token.splitlines())[start_idx10+1:start_idx11])  
        except Exception as e: 
            print(f"\n[예외처리]: {self.fname} 파일의 서식이 예상과 다릅니다. {e}")

        return contents_dict

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.


In [6]:
'''
#from data_collect.crawl_financeregulation import FinanceRegulationCrawler
'''

import os
import re
from urllib import request
from urllib.error import HTTPError
from bs4 import BeautifulSoup
import requests
import pandas as pd
#from data_collect.parse_HWPfile import FinanaceRegulation_HWP_Parser

class FinanceRegulationCrawler:
    '''
    return dict specification: (딕셔너리 구조(칼럼) 명세서)
        1. subject: [o] 상세 페이지 제목
        2. date: [o] 상세 페이지 작성일
        3. category1: [o] 대분류
        4. category2: [o] 중분류
        5. category3: [o] 예고일정
        6. contents: [x] 상세 페이지 contents (string text type, parsing 안되어있음, 주석처리 해둠)
        7. fname: [o] 상세페이지의 첨부파일 이름
        8. download_url: [o] 첨부파일 다운로드 링크
        9. attached_contents: [o] 첨부 파일의 contents (dict type, parsing 되어있음)
            9-1. 1.규제사무명
            9-2. 2.규제조문
            9-3. 3.위임법령
            9-4. 4.유형
            9-5. 5.입법예고
            9-6. 6.추진배경및정부개입필요성
            9-7. 7.규제내용
            9-8. 8.피규제집단및이해관계자
            9-9. 9.도입목표및기대효과
            9-10. 10.비용편익분석
    '''
    def __init__(self):
        self.site = "https://www.fsc.go.kr"
        self.download_tag = "/comm/getFile?"
        self.detail_tag = "/view?noticeId"

    def crawl_page(self, folder):
        self.folder = folder

        #999페이지까지 크롤링하여도 되나, 데모코드에서는 2페이지까지만 크롤링
        #max_page_num = 999 
        max_page_num = 2
        
        for page_num in range(1,max_page_num):
            page_param = f"/po040301?curPage={page_num}"
            # request 모듈을 사용하여 웹 페이지의 내용을 가져온다.
            url = self.site + page_param
            print(f"크롤링 진행 중 ... :{url}")
            r = requests.get(url)

            # beautiful soup 초기화
            soup = BeautifulSoup(r.text, "html.parser")
            items = soup.find_all("a")
            res = self.parse_the_page(items)

            if len(res) <= 0:
                print(f"실패: 페이지{page_num} 비어있음")
                break
            else:
                print(f"완료: 페이지{page_num}")
            
            yield res

    def download_attached_file(self, url, fname, directory):
        try:
            root = os.path.join(directory, fname)
            request.urlretrieve(url, root)

        except HTTPError as e:
            print(f"첨부파일 다운로드 실패, {url, fname}")
            return

    def parse_the_page(self, items):

        page_dict = dict()
        for item_idx, item in enumerate(items):
            if self.detail_tag in item.get("href"):
                item_dict = dict()
                detail_url = self.site + item.get('href')[1:]
                item_dict["detail_url"] = detail_url

                detail_r = requests.get(detail_url)
                detail_soup = BeautifulSoup(detail_r.text, "html.parser")

                subject =  detail_soup.select_one(".board-view-wrap > .header > .subject").get_text()
                day = detail_soup.select_one(".board-view-wrap > .header > .day").get_text().strip().split('\n')
                info = detail_soup.select_one(".board-view-wrap > .header > .info").get_text().strip().split('\n')
                contents = detail_soup.select_one(".board-view-wrap > .body > .cont").get_text().strip()
                
                item_dict["subject"] = subject
                item_dict["date"] = day[0]
                item_dict["cagegory1"] = info[0]
                item_dict["cagegory2"] = info[1]
                item_dict["cagegory3"] = info[2]
                #item_dict["contents"] = re.sub(r"[^a-zA-Z0-9가-힣.,\s\n]", "", contents)

                download_names = detail_soup.select(".body > .file > .file-list-wrap > .file-list > .name")
                download_urls  = detail_soup.select(".body > .file > .file-list-wrap > .file-list > .ico + .download")

                item_dict["fname"] = list()
                item_dict["download_url"] = list()
                item_dict["attached_contents"] = list()
                
                for item in zip(download_names, download_urls):
                    name = item[0].get_text().strip()
                    name = re.sub(r"[^a-zA-Z0-9가-힣._]", ' ', name).split()[:-2]
                    fname = ''.join(name)
                    if self.download_tag in item[1].find(href=True)['href']: #첨부파일 다운로드 링크가 있는 것들 중,
                        if re.search(r"hwp$", fname, re.IGNORECASE): #한글문서 첨부파일인 것들과
                            if re.search(r"규제영향분석서.hwp$", fname): #한글 파일의 이름에 '규제영향분석서'라는 문구가 있는 경우
                                download_url = self.site + item[1].find(href=True)['href']
                                self.download_attached_file(download_url, fname, self.folder)
                                item_dict["fname"].append(fname)
                                item_dict["download_url"].append(download_url)
                                hwp_handler = FinanaceRegulation_HWP_Parser(fname, self.folder)
                                hwp_res = hwp_handler.parse_from_text()
                                item_dict["attached_contents"].append(hwp_res)

                page_dict[item_idx] = item_dict

        return page_dict

## 1. 최신 규제정보 크롤링하여 수집
- `www.fsc.go.kr`, `https://ombudsman.kotra.or.kr/ob-kr/ob/i-2620/lgsltn/front/gv-list.do` 사이트 크롤링
- 크롤링 후, 규제영향분석 관련 정보 수집
- 수집 된 정보 파싱하여 주요 정보 추출 (dictionary)
 - 데모코드에서는 `page=1`만 서치하게끔 구성

In [7]:
fsc = FinanceRegulationCrawler()
res = fsc.crawl_page(folder="file")

regulation_candidate_list = list()
for r in res:
    regulation_candidate_list.append(r)

크롤링 진행 중 ... :https://www.fsc.go.kr/po040301?curPage=1
완료: 페이지1


In [8]:

regulation_list = list()

for page in regulation_candidate_list:
    for key, val in page.items():
        if val["fname"]: #규제영향 관련한 레포트가 있는 경우의 크롤링 결과를 출력
            regulation_list.append(val)



# 규제정보를 수집한 데이터 예시
regulation_df = pd.DataFrame(regulation_list)
print(regulation_df.loc[1])

detail_url           https://www.fsc.go.kr/po040301/view?noticeId=3810&curPage=1&srchKey=&srchText=&srchBeginDt=&srchEndDt=                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             

규제정보에서 가장 핵심적으로 중요한 필드는 다음과 같아요!

*   date  :  예고기간을 나타내는 기간
*   attached_contents  : 규제세부내용

attached_contents 에 해당하는 규제세부내용은 아래의 셀과 같습니다

In [9]:
regulation_content = pd.DataFrame()

for content in regulation_df["attached_contents"] : 
  regulation_content = regulation_content.append(content)

regulation_content.loc[0]      

,1.규제사무명,2.규제조문,3.위임법령,4.유형,5.입법예고,6.추진배경및정부개입필요성,7.규제내용,8.피규제집단및이해관계자,9.도입목표및기대효과,10.비용편익분석
0,감사인 지정 방식 변경,외부감사 및 회계 등에 관한 규정 제15조 별표3 및 별표4,주식회사 등의 외부감사에 관한 법률 시행령 제16조제4항,강화,2022.07.18 2022.08.26 규제의 필요성,현재 운영 중인 감사인 지정제도가 감사품질경쟁을 저하시키는 요인으로 작용되고 있어 감사품질과 역량에 상응하는 감사인 지정이 이루어질 수 있도록 기업과 감사인간 지정 방식을 개선하여 회계투명성 제고를 추진하기 위함 회계투명성 제고를 통한 이해관계자 보호를 위해 높은 수준의 외부감사가 필요한 지정대상기업은 감사품질이 우수한 회계법인이 지정감사를 수행하도록 할 필요,기업과 감사인의 지정 군 분류 체계를 개선하고 회계법인이 상위군으로 진입하기 위해서는 강화된 품질관리수준 등을 갖출 필요,피규제집단 회계법인 유 형 인원수 또는 규모 피규제자 회계법인 21년 지정신청 회계법인 58개사,규제의 적정성,기술영향평가 경쟁영향평가 중기영향평가 해당없음 해당없음 해당없음
0,민간중금리대출 금리상한 조정,상호금융업 감독규정 제4조의6,신용협동조합법 시행령 제16조의2제1호,신설,2022.07.13 2022.07.24 규제의 필요성,최근 한국은행 기준금리 인상 등에 따라 신규 신용대출 금리가 크게 상승 금리 상승으로 중저신용자 대출금리가 민간 중금리대출 금리 상한보다 높아지는 경우 금융회사가 금리상한에 맞추어 중저신용자 대출을 민간 중금리 대출로 취급할 유인 감소 - 이에 따라 중저신용자 신용대출 금리가 더욱 크게 상승할 가능성 금리상승이 민간중금리 금리상한 요건에 반영되도록 기준을 합리화하여 금융회사에 적절한 인센티브를 부여함으로써 민간 중금리 대출 확대를 유도할 필요,비조합원 대출한도 우대가 적용되는 민간중금리 대출 적격요건 중 금리요건을 기존의 민간중금리 금리상한100분의 8.5을 기준으로 조달금리 변동 폭을 고려하여 금융위원회가 정하는 금리상한으로 변경하며 이 경우에도 금리상한 한도는 100분의 10.5로 설정,상호금융업권,중금리대출 금리요건 합리화를 통해 금융회사에게 적절한 인센티브를 부여함으로써 기존의 민간중금리 대출 금리상한을 벗어나 고금리로 대출을 이용하던 중저신용자가 보다 낮은 중금리대 대출을 이용 규제의 적정성,단위백만원 비용 편익 순비용 피규제자 피규제자 이외 정성분석 조달금리 하락시 금융회사의 인센티브 축소 조달금리를 반영하는 민간중금리 금리요건 합리화로 중금리대출 활성화 주요내용 조달금리 상승시에는 민간중금리 대출 금리상한이 상승함에 따라 금융회사에게는 인센티브가 확대되고 고금리로 대출받던 중저신용자에게는 보다 낮은 금리로 대출을 받을 수 있는 기회가 증가 조달금리 하락시에는 민간중금리 대출 금리상한이 하락함에 따라 금융회사의 인센티브는 축소되나 이는 조달비용 하락에 따른 효과로서 금융회사에게 실질적 부담이 발생한다고 보기 어려운 측면이 있으며. 중저신용자는 낮은 금리로 중금리대출 활용 가능 민간중금리 기준에 조달금리 변경을 반영함으로써 금융회사와 중저신용자에 대한 순편익 증대
0,민간중금리대출 금리상한 조정,여신전문금융업감독규정 제5조의6,여신전문금융업법시행령 제17조제2항제6호,신설,2022.07.13 2022.07.24 규제의 필요성,최근 한국은행 기준금리 인상 등에 따라 신규 신용대출 금리가 크게 상승 금리 상승으로 중저신용자 대출금리가 민간 중금리대출 금리 상한보다 높아지는 경우 금융회사가 금리상한에 맞추어 중저신용자 대출을 민간 중금리 대출로 취급할 유인 감소 - 이에 따라 중저신용자 신용대출 금리가 더욱 크게 상승할 가능성 금리상승이 민간중금리 금리상한 요건에 반영되도록 기준을 합리화하여 금융회사에 적절한 인센티브를 부여함으로써 민간 중금리 대출 확대를 유도할 필요,본업자산 대비 대출자산 비중 산정시 80로 축소 반영되는 민간중금리 대출 적격요건 중 금리요건을 기존의 민간중금리 금리상한신용카드업자는 100분의 11.0 신용카드업자가 아닌 자는 100분의 14.0을 기준으로 조달금리 변동 폭을 고려하여 금융위원회가 정하는 금리상한으로 변경하며 이 경우에도 금리상한 한도는 신용카드업자는 100분의 13.0 신용카드업자가 아닌 자는 100분의 15.5로 설정,여신전문금융업권,중금리대출 금리요건 합리화를 통해 금융회사에게 적절한 인센티브를 부여함으로써 기존의 민간중금리 대출 금리상한을 벗어나 고금리로 대출을 이용하던 중저신용자가 보다 낮은 중금리대 대출을 이용 규제의 적정성,단위백만원 비용 편익 순비용 피규제자 피규제자 이외 정성분석 조달금리 하락시 금융회사의 인센티브 축소 조달금리를 반영하는 민간중금리 금리요건 합리화로 중금리대출 활성화 주요내용 조달금리 상승시에는 민간중금리 대출 금리상한이 상승함에 따라 금융회사에게는 인센티브가 확대되고 고금리로 대출받던 중저신용자에게는 보다 낮은 금리로 대출을 받을 수 있는 기회가 증가 조달금리 하락시에는 민간중금리 대출 금리상한이 하락함에 따라 금융회사의 인센티브는 축소되나 이는 조달비용 하락에 따른 효과로서 금융회사에게 실질적 부담이 발생한다고 보기 어려운 측면이 있으며. 중저신용자는 낮은 금리로 중금리대출 활용 가능 민간중금리 기준에 조달금리 변경을 반영함으로써 금융회사와 중저신용자에 대한 순편익 증대
0,민간중금리대출 금리상한 조정,상호저축은행업감독규정 제22조의2,상호저축은행법시행령제8조의2제1호,신설,2022.07.13 2022.07.24 규제의 필요성,최근 한국은행 기준금리 인상 등에 따라 신규 신용대출 금리가 크게 상승 금리 상승으로 중저신용자 대출금리가 민간 중금리대출 금리 상한보다 높아지는 경우 금융회사가 금리상한에 맞추어 중저신용자 대출을 민간 중금리 대출로 취급할 유인 감소 - 이에 따라 중저신용자 신용대출 금리가 더욱 크게 상승할 가능성 금리상승이 민간중금리 금리상한 요건에 반영되도록 기준을 합리화하여 금융회사에 적절한 인센티브를 부여함으로써 민간 중금리 대출 확대를 유도할 필요,영업구역내 신용공여 한도에 100분의 150으로 가산되어 적용되는 민간중금리 대출 적격요건 중 금리요건을 기존의 민간중금리 금리상한100분의 16을 기준으로 조달금리 변동 폭을 고려하여 금융위원회가 정하는 금리상한으로 변경하며 이 경우에도 금리상한 한도는 100분의 17.5로 설정,상호저축은행업권,중금리대출 금리요건 합리화를 통해 금융회사에게 적절한 인센티브를 부여함으로써 기존의 민간중금리 대출 금리상한을 벗어나 고금리로 대출을 이용하던 중저신용자가 보다 낮은 중금리대 대출을 이용 규제의 적정성,단위백만원 비용 편익 순비용 피규제자 피규제자 이외 정성분석 조달금리 하락시 금융회사의 인센티브 축소 조달금리를 반영하는 민간중금리 금리요건 합리화로 중금리대출 활성화 주요내용 조달금리 상승시에는 민간중금리 대출 금리상한이 상승함에 따라 금융회사에게는 인센티브가 확대되고 고금리로 대출받던 중저신용자에게는 보다 낮은 금리로 대출을 받을 수 있는 기회가 증가 조달금리 하락시에는 민간중금리 대출 금리상한이 하락함에 따라 금융회사의 인센티브는 축소되나 이는 조달비용 하락에 따른 효과로서 금융회사에게 실질적 부담이 발생한다고 보기 어려운 측면이 있으며. 중저신용자는 낮은 금리로 중금리대출 활용 가능 민간중금리 기준에 조달금리 변경을 반영함으로써 금융회사와 중저신용자에 대한 순편익 증대




> 규제 상세 정보를 보았을 때, 규제가 적용되는 [8.피규제집단및이해관계자]은 관련 제도에대한 공시 정보 알림을 받는다면 관련된 주식과 주종에 대한 정보를 빠르게 알 수 있을 거에요!

*   
[1.규제사무명] 동일한 경우에 유형, 입법예고, 추진배경및정부개입필요성, 
기대효과, 비용편익이 동일한 내용을 지님을 알 수 있음!

*   따라서 [1.규제사무명]을 기준으로 [5.입법예고]기간내에 [추진배경, 기대효과, 비용편익]에 대한 키워드를 추출하여 관련 기사내용을 검색한다면,

*   [8.피규제집단및이해관계자]들과 관련된 언론 정보에 대해 긍/부/중 에대한 정보 전달할 수 있을 것이며, 

*   [8.피규제집단및이해관계자]들에게 세부적으로 적용되는 [규제내용] 알림을 통해 정부 정책 변동에 대해 빠르게 전달 받아 대응 할 수 있을 것이다.

따라서 규제정보에 대한 언론 영향도를 파악하기 위한 최종적인 데이터셋은
다음과 같습니다

In [10]:
# attached_contents을 df 형태로 변환해서 데이터셋만들기
for idx in range(len(regulation_df)):
    content = regulation_df["attached_contents"].loc[idx]
    attached_contents_df = pd.DataFrame.from_dict(content)
    regulation_content = pd.concat([regulation_df,attached_contents_df ], axis=1)

# 필요없는 컬럼 삭제 
del regulation_content["detail_url"]
del regulation_content["attached_contents"]
del regulation_content["download_url"]

regulation_content.rename(columns={'cagegory3':'예고기간', "fname":"원본파일명" }, inplace = True)

print(regulation_content.loc[0])

subject           외부감사 및 회계 등에 관한 규정 일부개정고시(안) 규정변경예고                                                                                                                                                                                                                                                                                                                                                                          
date              2022-07-18                                                                                                                                                                                                                                                                                                                                                                                                   
cagegory1         구분규정변경예고                                                                                                                                              

규제관련 뉴스정보 검색을 위한 핵심 정보키워드 추출을 위해서 데이터셋 조합은 다음과 같습니다

*  [1.규제사무명]의 unique값 추출하기

1.   규제와 관련한 [피규제집단및이해관계자] 정리하여 추후 알림에 사용! 
2.   규제와 관련한 [검색정보] 정리하여 추후 언론 검색에 사용! 





In [11]:
regulationName_List = regulation_content['1.규제사무명'].unique()

# 1.규제사무명에 따른 8.피규제집단및이해관계자 모집    > 추후에 알림 할 때 사용됨
# 언론검색이후에 피규제집단에게 언론에 대한 정보를 제공하고, 세부적으로 다른 규제정보 알림을 보내기 위해 정리함

# Regulated groups and stakeholders 
regulation_content_groups = pd.DataFrame()
Regulated_groups = []

for regulationName in regulationName_List :
    print(regulationName)
    Regulated_groups = regulation_content[(regulation_content['1.규제사무명']==f"{regulationName}")]["8.피규제집단및이해관계자"]

    regulation_Regulated = { "1.규제사무명" : regulationName,
                              "Regulated_groups"  : Regulated_groups.unique()}

    regulation_content_groups = regulation_content_groups.append(regulation_Regulated,ignore_index=True)


regulation_content_groups

민간중금리대출 금리상한 조정
nan


,1.규제사무명,Regulated_groups
0,민간중금리대출 금리상한 조정,[상호저축은행업권]
1,NaN,[]


In [12]:
# 중복된 규제를 제거하여 관련 규제에 대한 검색데이터 처리를 간편화함
# regulation_for_search = regulation_content[]
#데이터프레임 중간 행,열 생략없이 모두 출력하는 설정
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', -1)

regulation_without_duplicates = regulation_content.drop_duplicates(subset=['1.규제사무명'])

print(regulation_without_duplicates)

                               subject        date cagegory1 cagegory2  \
0  외부감사 및 회계 등에 관한 규정 일부개정고시(안) 규정변경예고  2022-07-18  구분규정변경예고  법률구분행정규칙   
1  상호금융업감독규정 일부개정고시(안) 규정변경예고           2022-07-13  구분규정변경예고  법률구분행정규칙   

                          예고기간                     원본파일명          1.규제사무명  \
0  예고기간2022-07-18 ~ 2022-08-26  [3.외부감사규정규제영향분석서.hwp]     민간중금리대출 금리상한 조정   
1  예고기간2022-07-13 ~ 2022-07-24  [3.상호금융업감독규정규제영향분석서.hwp]  NaN               

               2.규제조문              3.위임법령 4.유형  \
0  상호저축은행업감독규정 제22조의2  상호저축은행법시행령제8조의2제1호  신설    
1  NaN                 NaN                 NaN   

                           5.입법예고  \
0  2022.07.13  2022.07.24 규제의 필요성   
1  NaN                              

                                                                                                                                                                                                                                                        6.추진배경및정부개입필요성  \
0    최근 한국은행 기준금

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  import sys


## 2. 규제정보 키워드를 활용하여 최신 뉴스 크롤링하여 수집
- 네이버 검색을 통한 뉴스 정보 크롤링
- 주요 정보 파싱하여 정보 추출, 정리(pd.DataFrame)
- 데모 시연을 위한 예제 선정 pick index as '1'
    - `selected_regulation = regulation_list[1]`

In [13]:
selected_regulation = regulation_list[1] # 데모 시연을 위한 예제 선정 pick index '1'

In [14]:
# from data_collect.crawl_navernews import NaverNEWSCrawler

# 크롤링시 필요한 라이브러리 불러오기
# -*- coding: UTF-8 -*-
from bs4 import BeautifulSoup
import requests
import re
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd

class NaverNEWSCrawler:

    def __init__(self, keyword, start_pg=1, end_pg=1):
        # 웹드라이버 설정
        #Colab에선 웹브라우저 창이 뜨지 않으므로 별도 설정한다.
        options = webdriver.ChromeOptions()
        options.add_argument('--headless')        # Head-less 설정
        options.add_argument('--no-sandbox')
        options.add_argument('--disable-dev-shm-usage')
        driver = webdriver.Chrome('chromedriver', options=options)

        # 검색어 입력
        self.search= keyword
        self.start_pg = start_pg
        self.end_pg = end_pg
        # naver url 생성
        self.search_urls = NaverNEWSCrawler.makeUrl(self.search, self.start_pg, self.end_pg)


        ## selenium으로 navernews만 뽑아오기##
        driver.implicitly_wait(3)
        # selenium으로 검색 페이지 불러오기 #
        naver_urls = []
        for search_url in self.search_urls:
            driver.get(search_url)
            time.sleep(1)  # 대기시간 변경 가능

            # 네이버 기사 눌러서 제목 및 본문 가져오기#
            # 네이버 기사가 있는 기사 css selector 모아오기
            search_news = driver.find_elements(By.CSS_SELECTOR, 'a.info')

            # 위에서 생성한 css selector list 하나씩 클릭하여 본문 url얻기
            for search_new in search_news:
                search_new.click()

                # 현재탭에 접근
                driver.switch_to.window(driver.window_handles[1])
                time.sleep(3)  # 대기시간 변경 가능

                # 네이버 뉴스 url만 가져오기
                url = driver.current_url

                if "news.naver.com" in url:
                    naver_urls.append(url)
                    print(url)
                else:
                    pass
                
                driver.close()
                driver.switch_to.window(driver.window_handles[0])
            
        ###naver 기사 본문 및 제목 가져오기###
        # ConnectionError방지
        headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) Chrome/98.0.4758.102"}

        titles = []
        contents = []
        news_dates = []

        for naver_url in naver_urls:
            original_html = requests.get(naver_url, headers=headers)
            html = BeautifulSoup(original_html.text, "html.parser")
            # 뉴스 제목 가져오기
            title = html.select("div#ct > div.media_end_head.go_trans > div.media_end_head_title > h2")
            # list합치기
            title = ''.join(str(title))
            # html태그제거
            pattern1 = '<[^>]*>'
            title = re.sub(pattern=pattern1, repl='', string=title)
            titles.append(title)

            # 뉴스 본문 가져오기
            content = html.select("div#dic_area")

            # 기사 텍스트만 가져오기
            # list합치기
            content = ''.join(str(content))
            # html태그제거 및 텍스트 다듬기
            content = re.sub(pattern=pattern1, repl='', string=content)
            pattern2 = """[\n\n\n\n\n// flash 오류를 우회하기 위한 함수 추가\nfunction _flash_removeCallback() {}"""
            content = content.replace(pattern2, '')
            contents.append(content)
                    
            # 날짜 가져오기
            html_date = html.select_one("div#ct> div.media_end_head.go_trans > div.media_end_head_info.nv_notrans > div.media_end_head_info_datestamp > div > span")
            if html_date != None:
                news_date = html_date.attrs['data-date-time']
                news_dates.append(news_date)
            else : 
                news_dates.append("None")
        driver.quit()
        # 데이터프레임으로 정리(titles,url,contents)

        self.news_df = pd.DataFrame({'date' : news_dates,
                        'title': titles, 
                        'content': contents,
                        'link': naver_urls, 
                        })

        self.news_df.to_csv('file/NaverNews_%s.csv' % self.search, index=False, encoding='utf-8-sig')
        return

    # 크롤링할 url 생성하는 함수 만들기(검색어, 크롤링 시작 페이지, 크롤링 종료 페이지)
    def makeUrl(search, start_pg, end_pg):
        if start_pg == end_pg:
            start_page = start_pg
            url = "https://search.naver.com/search.naver?where=news&sm=tab_pge&query=" + search + "&start=" + str(start_page)
            return [url]

        else:
            urls = []
            for i in range(start_pg, end_pg + 1):
                page = i
                url = "https://search.naver.com/search.naver?where=news&sm=tab_pge&query=" + search + "&start=" + str(page)
                urls.append(url)
            return urls

In [15]:
keyword = selected_regulation["attached_contents"][0]["1.규제사무명"] 
print(f"검색 키워드: {keyword}")

검색 키워드: 민간중금리대출 금리상한 조정


In [17]:
news = NaverNEWSCrawler(keyword=keyword, end_pg=3)
news.news_df

https://n.news.naver.com/mnews/article/001/0013316949?sid=101
https://n.news.naver.com/mnews/article/374/0000294986?sid=101
https://n.news.naver.com/mnews/article/014/0004859496?sid=101
https://n.news.naver.com/mnews/article/032/0003156340?sid=101
https://n.news.naver.com/mnews/article/081/0003283600?sid=101
https://n.news.naver.com/mnews/article/009/0004988140?sid=101
https://n.news.naver.com/mnews/article/001/0013316949?sid=101
https://n.news.naver.com/mnews/article/374/0000294986?sid=101
https://n.news.naver.com/mnews/article/014/0004859496?sid=101
https://n.news.naver.com/mnews/article/001/0013276638?sid=101
https://n.news.naver.com/mnews/article/081/0003283600?sid=101
https://n.news.naver.com/mnews/article/009/0004988140?sid=101
https://n.news.naver.com/mnews/article/374/0000294986?sid=101
https://n.news.naver.com/mnews/article/014/0004859496?sid=101
https://n.news.naver.com/mnews/article/001/0013276638?sid=101
https://n.news.naver.com/mnews/article/081/0003283600?sid=101
https://

,date,title,content,link
0,None,[대출금리 7% 땐 190만명 '최저 생계비 빼면' 원리금 못 갚는다],"[\n건보료만 내도 원리금 못 갚는 차주 120만명금융당국, 2금융권 중금리대출 상한선 올려 고금리 중·저신용자 이용 유도 \n\n\n\n서울시내 한 시중은행 대출 창구[연합뉴스 자료 사진] 2022.7.13 hama@yna.co.kr (서울=연합뉴스) 심재훈 이지헌 오주현 기자 = 가계 대출 평균 금리가 7% 수준에 이를 경우 소득에서 최저 생계비를 빼면 대출 원리금도 못 갚는 사람이 190만명에 달하는 것으로 파악됐다. 특히, 금리가 높은 대부업, 저축은행 등 2금융권과 자영업자의 다중 채무가 급증하는 것으로 파악돼 금융당국은 2금융권의 민간 중금리대출 상한선을 재조정하는 등 중·저신용자 보호에 적극적으로 나설 방침이다. 18일 금융권 등에 따르면 금융감독원이 최근 금리 급등기를 맞아 금리 상승이 가계 대출 차주의 상환 능력에 미치는 영향을 분석했더니 이런 결과가 나왔다.\n\n\n\n[그래픽] 금리 3%p 상승 시 가계대출자에 미치는 영향(서울=연합뉴스) 원형민 기자 = circlemin@yna.co.kr 페이스북 tuney.kr/LeYN1 트위터 @yonhap_graphics 올해 3월 말 현재 전체 금융권의 가계 대출은 1천616조2천억원으로, 평균 금리(3.96%)가 3%포인트 상승하면 대출자 1천646만명 중 총부채원리금상환비율(DSR)이 70%를 넘는 경우가 190만명에 달하는 것으로 분석됐다. 이는 기존 140만명에서 50만명이 늘어난 것이며 이들의 부채 금액은 357조5천억원에서 480조4천억원으로 122조9천억원이나 증가하게 된다.\n\n\n\n금리인상에 '영끌족' 이자부담 어쩌나(서울=연합뉴스) 신준희 기자 = 13일 서울시내 한 시중은행 대출 창구. 한국은행 금통위는 이날 기준금리를 한꺼번에 0.50%포인트 올렸다. 이에 따라 대출금리가 기준금리 인상 폭만큼만 올라도 가계대출자의 이자 부담은 24조원 가까이 불어날 것으로 추정된다. 2022.7.13 hama@yna.co.kr DSR이란 1년 동안 갚아야 하는 대출이자와 대출 원금이 소득과 비교해 얼마나 되는지를 계산한 수치다. DSR이 70%를 초과하는 경우 일반적으로 소득에서 최저 생계비를 제외했을 때 원리금을 감당하지 못하는 차주로 분류된다. 소득에서 소득세와 건강보험료 등만 차감해도 원리금을 못 갚는 대출자를 의미하는 DSR 90% 초과 차주는 평균 금리(3.96%)가 3%포인트 상승하면 90만명에서 120만명으로 30만명 증가하는 것으로 파악됐다. 이들의 부채 금액은 254조원에서 336조원으로 늘어나게 된다. 이럴 경우 DSR 90% 초과 차주 비중은 2금융권이 8.4%(62만명)에서 10.3%(76만명), 자영업자는 10.2%(21만9천명)에서 13%(28만명)로 각각 늘게 된다. 다중 채무자 중 DSR 90% 초과 차주 비중은 8.7%(33만2천명)에서 12%(45만6천명)로 급증하게 된다. 금융시장에서는 미국 연방준비제도(Fed·연준)가 강도 높은 긴축을 이어가고 있고 최근 빅스텝(한번에 기준금리 0.50%포인트 인상)을 단행한 한국은행도 계속해서 기준금리를 올릴 것으로 예상돼 시장 금리 상승도 가속할 것으로 예측하고 있다. 금융위원회와 금감원 등 금융 당국은 이런 금리 상승기를 맞아 2금융권에서 대출이 많은 자영업자의 연착륙 유도를 적극적으로 추진할 방침이다. 금융당국은 중·저신용자에 대한 자금 공급을 원활히 하기 위해 2016년부터 중금리 대출 제도를 운용하고 있다. 중금리 대출 공급액은 2016년 1조3천억원에서 지난해 21조5천억원으로 급증했다. \n\n\n\n서울시내 한 시중은행 대출 창구[연합뉴스 자료사진] 2022.7.13 hama@yna.co.kr이 가운데 민간 중금리 대출은 신용 하위 50%인 개인 대출자를 위한 제도로, 업권별 금리 상한 요건을 충족하는 비보증부 신용대출에 대해 인센티브를 부여하는 방식으로 운영하고 있다. 금융위는 최근 민간 중금리 대출의 금리 상한 기준을 합리화하는 내용의 상호금융업·저축은행·여신전문금융업 감독규정 일부 개정 고시의 규정 변경을 예고했다. 이는 최근 금리가 급등하는 가운데 고금리로 대출을 이용하던 중·저신용자가 보다 낮은 중금리대 대출을 이용할 수 있도록 민간 중금리 금리상한 기준을 합리화하기 위한 조치다. 이에 따라 상호금융의 경우 민간 중금리 대출의 금리 상한을 8.5%에서 10.5%로 상향 조정하기로 했다. 여신전문금융업은 신용카드업의 경우 민간 중금리 대출의 금리 상한을 11%에서 13%, 신용카드 외 사업자는 14%에서 15.5%로 설정할 예정이다. 저축은행은 16%에서 17.5%로 올릴 방침이다. 금융당국 관계자는 ""최근 기준금리 인상 등에 따라 신규 신용대출 금리가 크게 올랐다""면서 ""중·저신용자의 기존 대출 금리가 민간 중금리 대출 금리 상한보다 높아지는 경우 금융사는 중·저신용자 대출을 민간 중금리 대출로 취급할 이유가 줄어들게 된다""고 말했다. 그는 ""이렇게 되면 중·저신용자 신용대출 금리가 급등할 가능성이 있다""면서 ""이번 조치는 최근의 금리 상승분을 민간 중금리 대출의 금리 상한 요건에 반영해 금융사에 적절한 인센티브를 부여함으로써 민간 중금리 대출 확대를 유도하기 위한 것""이라고 덧붙였다. president21@yna.co.kr pan@yna.co.kr viva5@yna.co.kr\n]",https://n.news.naver.com/mnews/article/001/0013316949?sid=101
1,None,[대출금리 7% 땐 190만명 원리금 못 갚아],"[\n\t\t\t[앵커] 이렇게 다양한 대응 방안이 시도되고 있지만, 금리 인상 자체를 막을 순 없겠죠.지금처럼 금리가 올라서 가계대출의 평균금리가 7%에 도달하면, 190만명에 달하는 사람이 최저 생계비를 빼면 원리금도 못 갚는 것으로 파악됐습니다.엄하은 기자, 정확히 어떤 통계가 나온 건가요?[기자]총부채원리금상환비율인 DSR이 70%를 초과하면, 일반적으로 소득에서 최저 생계비를 뺐을 때 원리금을 감당하지 못하는 차주로 분류됩니다.금융권에 따르면 올해 3월 말 기준 전체 금융권 가계 대출은 1616조 2천억원인데요.3.96%인 평균 금리가 3%p 상승하면 대출자 1646만명 중 DSR이 70%를 초과하는 경우가 190만명에 달하는 것으로 분석됐습니다.기존 140만명에서 50만명이 늘어난 것인데, 이들의 부채 금액은 122조 9천억원 늘어, 480조 4천억원으로 증가하게 됩니다.소득에서 소득세와 건강보험료 등만 차감해도 원리금을 못 갚는 대출자를 의미하는 DSR 90% 초과 차주들도 30만명 증가하는 것으로 파악됐는데요.특히, 금리가 높은 대부업, 저축은행 등 2금융권과 자영업자의 다중 채무가 급증하는 것으로 나타났습니다.[앵커]문제는 앞으로 금리가 더 오른다는 거잖아요?[기자]미국 연방준비제도가 강도 높은 긴축을 이어가고 있고, 여기에 맞춰 한국은행도 기준금리를 올릴 거라 시중금리 상승은 더 가팔라질 것으로 보입니다.금융당국은 2금융권에서 대출이 많은 자영업자 등의 연착륙을 유도할 방침입니다.중·저신용자가 보다 낮은 중금리대 대출을 이용할 수 있도록 민간 중금리 금리상한 기준을 합리화하기 위한 조치에 나섰는데요.이에 따라 상호금융은 민간 중금리 대출의 금리 상한을 8.5%에서 10.5%로 상향 조정하기로 했습니다.SBS Biz 엄하은입니다. \n\t\t]",https://n.news.naver.com/mnews/article/374/0000294986?sid=101
2,None,[금리상승에 중금리 상한 합리화..중금리 대출 확대 유도],"[\n\t\t\t [파이낸셜뉴스] \n\n\n\n2022년 하반기 민간중금리 대출 금리상한 요건 사진=뉴시스 금리가 급격히 오르자 금융당국이 금융권 중금리 대출 금리 상한 요건 산정 방식을 변경하기로 했다. 금리 상승이 중금리 금리상한 요건에 반영되도록 기준을 합리화해 중금리 대출 확대를 유도하겠다는 것이다. 29일 금융위원회는 '민간중금리 대출 금리요건 개선방안'을 통해 매반기 조달금리 변동 폭을 근거로 민간중금리대출 금리 

## 3. 규제정보와 뉴스데이터 전처리
- 파싱 결과 얻어진 규제정보와 뉴스데이터 전처리(클린 텍스트)
- 형태소 추출(예선: 한나눔 사전 사용하여 한국어 형태소 분석 추출)
- Filtering Noun: Using Frequent Value and TF-IDF sparse matrix 
    - vectorizer = TfidfVectorizer()
    - embeddings = vectorizer.fit_transform(docs)

In [18]:
# from data_preprocessing.preprocess_text import TextCleaner, NounExtracter

import re
import unicodedata
from konlpy.tag import Hannanum
from pynori.korean_analyzer import KoreanAnalyzer
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd

class TextCleaner:
    def remove_unicode(input_str):
        # 유니코드 :  https://gist.github.com/Pusnow/aa865fa21f9557fa58d691a8b79f8a6d
        try:
            input_str = str(input_str)
            
        except (TypeError, NameError): # unicode is a default on python 3 
             pass

        text = unicodedata.normalize('NFKC', input_str).encode('utf-8','ignore').decode("utf-8")
            
        return text

    def clean_ko_sentences(sentences):
        sentence = TextCleaner.remove_unicode(sentences)
        # 정규 표현식 필터
        RE_FILTER = re.compile(r"[#▲,<>!?\"'·:;=~()]") # [] 안에 제거할 정규표현식을 추가한다
        # 특수기호 제거
        sentences = re.sub(RE_FILTER, "", sentence)
        sentences = re.sub(r"[^A-Za-z0-9가-힣?!,.¿]+", " ",sentences) 
        sentences = re.sub(r"\s"," ", sentences)# \n도 공백으로 대체해줌
        sentences = sentences.strip()  
        
        return str(sentences)
    
    def clean_ko_nouns(nounsList):
        nouns_list = []

        for noun in nounsList:
            if len(noun) > 1: # 한 글자 이하일때 삭제
                p = re.compile(r"[^0-9]+") # 숫자로 시작되는 글자 삭제
                noun = p.match(noun)
                if noun:
                    nouns_list.append(noun.group())
        
        return nouns_list

    def html_delete(raw_review):
        # HTML 코드 제거
        clean_text = BeautifulSoup(raw_review, "html.parser").get_text()
        return clean_text

class NounExtracter:

    def Morph_ko_document(sentence_List, database = "hannanum"):
        # KoNLPy 형태소분석기 설정 '
        # 형태소 분석기 적용부분 
        nouns = []

        if database == "hannanum":
            analyzer = Hannanum()
            for sentence in sentence_List:
                sentence = TextCleaner.clean_ko_sentences(sentence)
                nouns_list = analyzer.nouns(str(sentence))
                nouns_list=TextCleaner.clean_ko_nouns(nouns_list)
                nouns.append(nouns_list)

        elif database == "nori":
            analyzer = KoreanAnalyzer(
              decompound_mode='DISCARD', # DISCARD or MIXED or NONE
              infl_decompound_mode='DISCARD', # DISCARD or MIXED or NONE
              discard_punctuation=True,
              output_unknown_unigrams=False,
              pos_filter=False, stop_tags=['JKS', 'JKB', 'VV', 'EF'],
              synonym_filter=False, mode_synonym='NORM', # NORM or EXTENSION
            ) 
            for sentence in sentence_List:
                sentence = TextCleaner.clean_ko_sentences(sentence)
                # print(sentence)
                nouns_list = analyzer.do_analysis(str(sentence))['termAtt']
                nouns_list=TextCleaner.clean_ko_nouns(nouns_list)
                nouns.append(nouns_list)

        return nouns 

### 3-1. 자연어 전처리
- corpus, noun 추출

3-1-1. 뉴스 데이터 전처리(클린텍스트, 형태소(명사) 추출)

In [19]:
nouns_candidate_per_news = NounExtracter.Morph_ko_document(news.news_df["content"], database="hannanum")
nouns_per_news = list()
for nouns in nouns_candidate_per_news:
    nouns_per_news.append(nouns)
nouns_df = pd.DataFrame({"nouns": nouns_per_news})

In [20]:
X_news_contents = pd.concat([news.news_df, nouns_df], axis=1)
X_news_contents

,date,title,content,link,nouns
0,None,[대출금리 7% 땐 190만명 '최저 생계비 빼면' 원리금 못 갚는다],"[\n건보료만 내도 원리금 못 갚는 차주 120만명금융당국, 2금융권 중금리대출 상한선 올려 고금리 중·저신용자 이용 유도 \n\n\n\n서울시내 한 시중은행 대출 창구[연합뉴스 자료 사진] 2022.7.13 hama@yna.co.kr (서울=연합뉴스) 심재훈 이지헌 오주현 기자 = 가계 대출 평균 금리가 7% 수준에 이를 경우 소득에서 최저 생계비를 빼면 대출 원리금도 못 갚는 사람이 190만명에 달하는 것으로 파악됐다. 특히, 금리가 높은 대부업, 저축은행 등 2금융권과 자영업자의 다중 채무가 급증하는 것으로 파악돼 금융당국은 2금융권의 민간 중금리대출 상한선을 재조정하는 등 중·저신용자 보호에 적극적으로 나설 방침이다. 18일 금융권 등에 따르면 금융감독원이 최근 금리 급등기를 맞아 금리 상승이 가계 대출 차주의 상환 능력에 미치는 영향을 분석했더니 이런 결과가 나왔다.\n\n\n\n[그래픽] 금리 3%p 상승 시 가계대출자에 미치는 영향(서울=연합뉴스) 원형민 기자 = circlemin@yna.co.kr 페이스북 tuney.kr/LeYN1 트위터 @yonhap_graphics 올해 3월 말 현재 전체 금융권의 가계 대출은 1천616조2천억원으로, 평균 금리(3.96%)가 3%포인트 상승하면 대출자 1천646만명 중 총부채원리금상환비율(DSR)이 70%를 넘는 경우가 190만명에 달하는 것으로 분석됐다. 이는 기존 140만명에서 50만명이 늘어난 것이며 이들의 부채 금액은 357조5천억원에서 480조4천억원으로 122조9천억원이나 증가하게 된다.\n\n\n\n금리인상에 '영끌족' 이자부담 어쩌나(서울=연합뉴스) 신준희 기자 = 13일 서울시내 한 시중은행 대출 창구. 한국은행 금통위는 이날 기준금리를 한꺼번에 0.50%포인트 올렸다. 이에 따라 대출금리가 기준금리 인상 폭만큼만 올라도 가계대출자의 이자 부담은 24조원 가까이 불어날 것으로 추정된다. 2022.7.13 hama@yna.co.kr DSR이란 1년 동안 갚아야 하는 대출이자와 대출 원금이 소득과 비교해 얼마나 되는지를 계산한 수치다. DSR이 70%를 초과하는 경우 일반적으로 소득에서 최저 생계비를 제외했을 때 원리금을 감당하지 못하는 차주로 분류된다. 소득에서 소득세와 건강보험료 등만 차감해도 원리금을 못 갚는 대출자를 의미하는 DSR 90% 초과 차주는 평균 금리(3.96%)가 3%포인트 상승하면 90만명에서 120만명으로 30만명 증가하는 것으로 파악됐다. 이들의 부채 금액은 254조원에서 336조원으로 늘어나게 된다. 이럴 경우 DSR 90% 초과 차주 비중은 2금융권이 8.4%(62만명)에서 10.3%(76만명), 자영업자는 10.2%(21만9천명)에서 13%(28만명)로 각각 늘게 된다. 다중 채무자 중 DSR 90% 초과 차주 비중은 8.7%(33만2천명)에서 12%(45만6천명)로 급증하게 된다. 금융시장에서는 미국 연방준비제도(Fed·연준)가 강도 높은 긴축을 이어가고 있고 최근 빅스텝(한번에 기준금리 0.50%포인트 인상)을 단행한 한국은행도 계속해서 기준금리를 올릴 것으로 예상돼 시장 금리 상승도 가속할 것으로 예측하고 있다. 금융위원회와 금감원 등 금융 당국은 이런 금리 상승기를 맞아 2금융권에서 대출이 많은 자영업자의 연착륙 유도를 적극적으로 추진할 방침이다. 금융당국은 중·저신용자에 대한 자금 공급을 원활히 하기 위해 2016년부터 중금리 대출 제도를 운용하고 있다. 중금리 대출 공급액은 2016년 1조3천억원에서 지난해 21조5천억원으로 급증했다. \n\n\n\n서울시내 한 시중은행 대출 창구[연합뉴스 자료사진] 2022.7.13 hama@yna.co.kr이 가운데 민간 중금리 대출은 신용 하위 50%인 개인 대출자를 위한 제도로, 업권별 금리 상한 요건을 충족하는 비보증부 신용대출에 대해 인센티브를 부여하는 방식으로 운영하고 있다. 금융위는 최근 민간 중금리 대출의 금리 상한 기준을 합리화하는 내용의 상호금융업·저축은행·여신전문금융업 감독규정 일부 개정 고시의 규정 변경을 예고했다. 이는 최근 금리가 급등하는 가운데 고금리로 대출을 이용하던 중·저신용자가 보다 낮은 중금리대 대출을 이용할 수 있도록 민간 중금리 금리상한 기준을 합리화하기 위한 조치다. 이에 따라 상호금융의 경우 민간 중금리 대출의 금리 상한을 8.5%에서 10.5%로 상향 조정하기로 했다. 여신전문금융업은 신용카드업의 경우 민간 중금리 대출의 금리 상한을 11%에서 13%, 신용카드 외 사업자는 14%에서 15.5%로 설정할 예정이다. 저축은행은 16%에서 17.5%로 올릴 방침이다. 금융당국 관계자는 ""최근 기준금리 인상 등에 따라 신규 신용대출 금리가 크게 올랐다""면서 ""중·저신용자의 기존 대출 금리가 민간 중금리 대출 금리 상한보다 높아지는 경우 금융사는 중·저신용자 대출을 민간 중금리 대출로 취급할 이유가 줄어들게 된다""고 말했다. 그는 ""이렇게 되면 중·저신용자 신용대출 금리가 급등할 가능성이 있다""면서 ""이번 조치는 최근의 금리 상승분을 민간 중금리 대출의 금리 상한 요건에 반영해 금융사에 적절한 인센티브를 부여함으로써 민간 중금리 대출 확대를 유도하기 위한 것""이라고 덧붙였다. president21@yna.co.kr pan@yna.co.kr viva5@yna.co.kr\n]",https://n.news.naver.com/mnews/article/001/0013316949?sid=101,"[건보료만, 원리금, 차주, 중금리대출, 상한선, 고금리, 중저신용자, 이용, 유도, 서울시내, 시중은행, 대출, 창구, 연합뉴스, 자료, 서울연합뉴스, 심재훈, 이지헌, 오주현, 가계, 대출, 평균, 금리, 수준, 경우, 소득, 최저, 생계비, 대출, 원리금, 사람, 파악, 금리, 대부업, 저축은행, 자영업자, 다중, 채무, 급증, 파악, 금융당국, 민간, 중금리대출, 상한선, 재조정, 중저신용자, 보호, 적극적, 방침, 금융권, 금융감독원, 금리, 급등기, 금리, 상승, 가계, 대출, 차주, 상환, 능력, 영향, 분석, 결과, 그래픽, 금리, 상승, 가계대출자, 영향서울연합뉴스, 원형민, 페이스북, LeYN, 트위터, 올해, 현재, 전체, 금융권, 가계, 대출, 평균, 금리, 포인트, 상승, 대출자, 총부채원리금상환비율DSR, 경우, 분석, 기존, 부채, 금액, 증가, 금리인상, 영끌족, 이자부담, 어쩌나서울연합뉴스, 신준희, 서울시내, 시중은행, 대출, 창구, 한국은행, ...]"
1,None,[대출금리 7% 땐 190만명 원리금 못 갚아],"[\n\t\t\t[앵커] 이렇게 다양한 대응 방안이 시도되고 있지만, 금리 인상 자체를 막을 순 없겠죠.지금처럼 금리가 올라서 가계대출의 평균금리가 7%에 도달하면, 190만명에 달하는 사람이 최저 생계비를 빼면 원리금도 못 갚는 것으로 파악됐습니다.엄하은 기자, 정확히 어떤 통계가 나온 건가요?[기자]총부채원리금상환비율인 DSR이 70%를 초과하면, 일반적으로 소득에서 최저 생계비를 뺐을 때 원리금을 감당하지 못하는 차주로 분류됩니다.금융권에 따르면 올해 3월 말 기준 전체 금융권 가계 대출은 1616조 2천억원인데요.3.96%인 평균 금리가 3%p 상승하면 대출자 1646만명 중 DSR이 70%를 초과하는 경우가 190만명에 달하는 것으로 분석됐습니다.기존 140만명에서 50만명이 늘어난 것인데, 이들의 부채 금액은 122조 9천억원 늘어, 480조 4천억원으로 증가하게 됩니다.소득에서 소득세와 건강보험료 등만 차감해도 원리금을 못 갚는 대출자를 의미하는 DSR 90% 초과 차주들도 30만명 증가하는 것으로 파악됐는데요.특히, 금리가 높은 대부업, 저축은행 등 2금융권과 자영업자의 다중 채무가 급증하는 것으로 나타났습니다.[앵커]문제는 앞으로 금리가 더 오른다는 거잖아요?[기자]미국 연방준비제도가 강도 높은 긴축을 이어가고 있고, 여기에 맞춰 한국은행도 기준금리를 올릴 거라 시중금리 상승은 더 가팔라질 것으로 보입니다.금융당국은 2금융권에서 대출이 많은 자영업자 등의

3-1-2. 규제정보 데이터 전처리(클린텍스트, 형태소(명사) 추출)

In [21]:
regulation_contents = pd.concat((pd.DataFrame(selected_regulation) , pd.DataFrame(selected_regulation["attached_contents"])), axis=1).drop("attached_contents", axis=1)
#regulation_contents[regulation_contents.columns[-10:-1]]
nouns_candidate_in_regulation = NounExtracter.Morph_ko_document(regulation_contents[["6.추진배경및정부개입필요성", "7.규제내용", "9.도입목표및기대효과", "10.비용편익분석"]].transpose()[0])
nouns_in_regulation = list()
for nouns in nouns_candidate_in_regulation:
    nouns_in_regulation.append(nouns)
nouns_df = pd.DataFrame({"nouns": nouns_in_regulation})
nouns_df=nouns_df.transpose().rename(columns={0: "6.추진배경및정부개입필요성_nouns", 1: "7.규제내용_nouns", 2: "9.도입목표및기대효과_nouns", 3: "10.비용편익분석_nouns"}).reset_index(drop=True)

In [22]:
X_regulation_contents = pd.concat([regulation_contents, nouns_df], axis=1)
X_regulation_contents

,detail_url,subject,date,cagegory1,cagegory2,cagegory3,fname,download_url,1.규제사무명,2.규제조문,3.위임법령,4.유형,5.입법예고,6.추진배경및정부개입필요성,7.규제내용,8.피규제집단및이해관계자,9.도입목표및기대효과,10.비용편익분석,6.추진배경및정부개입필요성_nouns,7.규제내용_nouns,9.도입목표및기대효과_nouns,10.비용편익분석_nouns
0,https://www.fsc.go.kr/po040301/view?noticeId=3810&curPage=1&srchKey=&srchText=&srchBeginDt=&srchEndDt=,상호금융업감독규정 일부개정고시(안) 규정변경예고,2022-07-13,구분규정변경예고,법률구분행정규칙,예고기간2022-07-13 ~ 2022-07-24,3.상호금융업감독규정규제영향분석서.hwp,https://www.fsc.go.kr/comm/getFile?srvcId=RULENOTICE&upperNo=3810&fileTy=ATTACH&fileNo=4,민간중금리대출 금리상한 조정,상호금융업 감독규정 제4조의6,신용협동조합법 시행령 제16조의2제1호,신설,2022.07.13 2022.07.24 규제의 필요성,최근 한국은행 기준금리 인상 등에 따라 신규 신용대출 금리가 크게 상승 금리 상승으로 중저신용자 대출금리가 민간 중금리대출 금리 상한보다 높아지는 경우 금융회사가 금리상한에 맞추어 중저신용자 대출을 민간 중금리 대출로 취급할 유인 감소 - 이에 따라 중저신용자 신용대출 금리가 더욱 크게 상승할 가능성 금리상승이 민간중금리 금리상한 요건에 반영되도록 기준을 합리화하여 금융회사에 적절한 인센티브를 부여함으로써 민간 중금리 대출 확대를 유도할 필요,비조합원 대출한도 우대가 적용되는 민간중금리 대출 적격요건 중 금리요건을 기존의 민간중금리 금리상한100분의 8.5을 기준으로 조달금리 변동 폭을 고려하여 금융위원회가 정하는 금리상한으로 변경하며 이 경우에도 금리상한 한도는 100분의 10.5로 설정,상호금융업권,중금리대출 금리요건 합리화를 통해 금융회사에게 적절한 인센티브를 부여함으로써 기존의 민간중금리 대출 금리상한을 벗어나 고금리로 대출을 이용하던 중저신용자가 보다 낮은 중금리대 대출을 이용 규제의 적정성,단위백만원 비용 편익 순비용 피규제자 피규제자 이외 정성분석 조달금리 하락시 금융회사의 인센티브 축소 조달금리를 반영하는 민간중금리 금리요건 합리화로 중금리대출 활성화 주요내용 조달금리 상승시에는 민간중금리 대출 금리상한이 상승함에 따라 금융회사에게는 인센티브가 확대되고 고금리로 대출받던 중저신용자에게는 보다 낮은 금리로 대출을 받을 수 있는 기회가 증가 조달금리 하락시에는 민간중금리 대출 금리상한이 하락함에 따라 금융회사의 인센티브는 축소되나 이는 조달비용 하락에 따른 효과로서 금융회사에게 실질적 부담이 발생한다고 보기 어려운 측면이 있으며. 중저신용자는 낮은 금리로 중금리대출 활용 가능 민간중금리 기준에 조달금리 변경을 반영함으로써 금융회사와 중저신용자에 대한 순편익 증대,"[한국은행, 기준금리, 인상, 신규, 신용대출, 금리, 상승, 금리, 상승, 중저신용자, 대출금리, 민간, 중금리대출, 금리, 상한, 경우, 금융회사, 금리상한, 중저신용자, 대출, 민간, 중금리, 대출, 취급, 감소, 중저신용자, 신용대출, 금리, 상승, 가능성, 금리상승, 민간중금리, 금리상한, 요건, 반영, 기준, 합리, 금융회사, 적절, 인센티브, 민간, 중금리, 대출, 확대, 유도, 필요]","[조합원, 대출한도, 우대, 적용, 민간중금리, 대출, 적격요건, 금리요건, 기존, 민간중금리, 금리상한, 기준, 조달금리, 변동, 금융위원회, 금리상한, 변경, 경우, 금리상한, 설정]","[중금리대출, 금리요건, 합리화, 금융회사, 적절, 인센티브, 기존, 민간중금리, 대출, 금리상한, 고금리, 대출, 이용, 중저신용자, 중금리대, 대출, 이용, 규제, 적정성]","[단위백만원, 비용, 편익, 순비용, 피규제, 피규제, 이외, 정성분석, 조달금리, 하락시, 금융회사, 인센티브, 축소, 조달금리, 반영, 민간중금리, 금리요건, 합리화, 중금리대출, 활성화, 주요내용, 조달금리, 상승시, 민간중금리, 대출, 금리상한, 상승, 금융회사, 인센티브, 확대, 고금리, 대출, 중저신용자에게, 금리, 대출, 기회, 조달금리, 하락시, 민간중금리, 대출, 금리상한, 하락, 금융회사, 인센티브, 축소, 조달비용, 하락, 효과, 금융회사, 실질적, 부담, 발생, 측면, 중저신용자, 금리, 중금리대출, 활용, 가능, 민간중금리, 기준, 조달금리, 변경, 반영함, 금융회사, 중저신용자, 순편익, 증대]"


- 추출된 형태소(명사) 중 중요 명사만을 필터링
    - using `TF-IDF`, `Frequent Value`

In [23]:
'''
# data_preprocessing.filter_noun.py
'''

from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
import pandas as pd
# from data_preprocessing.preprocess_text import NounExtracter

class NounsFilter_Tfidf_and_Freq:
    def __init__(self, input_corpus):
        self.input_corpus = input_corpus.dropna() # one sentence

        self.get_frequency()
        self.get_tfidf()

    def get_frequency(self):
        self.nouns = NounExtracter.Morph_ko_document(self.input_corpus, database = "hannanum")

        counter = dict()
        for nouns in self.nouns:
            for noun in nouns:
                if noun in counter.keys():
                    counter[noun] += 1
                else:
                    counter[noun] =1

        self.counter_dict = pd.DataFrame(counter, index=["freq"])
        return self.counter_dict
    
    def get_tfidf(self):
        self.vectorizer = TfidfVectorizer()
        self.tfidf_value = self.vectorizer.fit_transform(self.input_corpus)

        self.make_tfidf_dict()
        self.get_NoMeaningRate()
        self.get_tfidf_feature_names()

        return self.tfidf_value 


    def make_tfidf_dict(self):
        word = self.vectorizer.inverse_transform(self.tfidf_value)
        value = (self.tfidf_value).data

        self.tfidf_dict = pd.DataFrame(data=value).transpose()
        self.tfidf_dict.index = ["value"]
        self.tfidf_dict.columns = word[0]

        return self.tfidf_dict

    def get_tfidf_feature_names(self):
        self.tfidf_unique_word = self.vectorizer.get_feature_names() # 유니크한 단어 목록
        return self.tfidf_unique_word

    def get_NoMeaningRate(self):
        NMR = (self.counter_dict.loc["freq"] - self.tfidf_dict.loc["value"]) / self.counter_dict.loc["freq"]  #NMR = NoMeaningRate
        NMR.dropna(inplace=True)
        NMR=NMR.sort_values(ascending=True)

        self.NMR = NMR
        return self.NMR 

    def pick_the_best_K_nouns(self, K):
        return self.NMR.head(K).index

In [24]:
best_nouns= list()
for idx, series in X_news_contents["content"].iteritems():
    filter = NounsFilter_Tfidf_and_Freq(pd.Series(series))
    best_nouns.append(filter.pick_the_best_K_nouns(K=10).values)
    
X_news_contents["best_nouns"] = best_nouns
X_news_contents

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,date,title,content,link,nouns,best_nouns
0,None,[대출금리 7% 땐 190만명 '최저 생계비 빼면' 원리금 못 갚는다],"[\n건보료만 내도 원리금 못 갚는 차주 120만명금융당국, 2금융권 중금리대출 상한선 올려 고금리 중·저신용자 이용 유도 \n\n\n\n서울시내 한 시중은행 대출 창구[연합뉴스 자료 사진] 2022.7.13 hama@yna.co.kr (서울=연합뉴스) 심재훈 이지헌 오주현 기자 = 가계 대출 평균 금리가 7% 수준에 이를 경우 소득에서 최저 생계비를 빼면 대출 원리금도 못 갚는 사람이 190만명에 달하는 것으로 파악됐다. 특히, 금리가 높은 대부업, 저축은행 등 2금융권과 자영업자의 다중 채무가 급증하는 것으로 파악돼 금융당국은 2금융권의 민간 중금리대출 상한선을 재조정하는 등 중·저신용자 보호에 적극적으로 나설 방침이다. 18일 금융권 등에 따르면 금융감독원이 최근 금리 급등기를 맞아 금리 상승이 가계 대출 차주의 상환 능력에 미치는 영향을 분석했더니 이런 결과가 나왔다.\n\n\n\n[그래픽] 금리 3%p 상승 시 가계대출자에 미치는 영향(서울=연합뉴스) 원형민 기자 = circlemin@yna.co.kr 페이스북 tuney.kr/LeYN1 트위터 @yonhap_graphics 올해 3월 말 현재 전체 금융권의 가계 대출은 1천616조2천억원으로, 평균 금리(3.96%)가 3%포인트 상승하면 대출자 1천646만명 중 총부채원리금상환비율(DSR)이 70%를 넘는 경우가 190만명에 달하는 것으로 분석됐다. 이는 기존 140만명에서 50만명이 늘어난 것이며 이들의 부채 금액은 357조5천억원에서 480조4천억원으로 122조9천억원이나 증가하게 된다.\n\n\n\n금리인상에 '영끌족' 이자부담 어쩌나(서울=연합뉴스) 신준희 기자 = 13일 서울시내 한 시중은행 대출 창구. 한국은행 금통위는 이날 기준금리를 한꺼번에 0.50%포인트 올렸다. 이에 따라 대출금리가 기준금리 인상 폭만큼만 올라도 가계대출자의 이자 부담은 24조원 가까이 불어날 것으로 추정된다. 2022.7.13 hama@yna.co.kr DSR이란 1년 동안 갚아야 하는 대출이자와 대출 원금이 소득과 비교해 얼마나 되는지를 계산한 수치다. DSR이 70%를 초과하는 경우 일반적으로 소득에서 최저 생계비를 제외했을 때 원리금을 감당하지 못하는 차주로 분류된다. 소득에서 소득세와 건강보험료 등만 차감해도 원리금을 못 갚는 대출자를 의미하는 DSR 90% 초과 차주는 평균 금리(3.96%)가 3%포인트 상승하면 90만명에서 120만명으로 30만명 증가하는 것으로 파악됐다. 이들의 부채 금액은 254조원에서 336조원으로 늘어나게 된다. 이럴 경우 DSR 90% 초과 차주 비중은 2금융권이 8.4%(62만명)에서 10.3%(76만명), 자영업자는 10.2%(21만9천명)에서 13%(28만명)로 각각 늘게 된다. 다중 채무자 중 DSR 90% 초과 차주 비중은 8.7%(33만2천명)에서 12%(45만6천명)로 급증하게 된다. 금융시장에서는 미국 연방준비제도(Fed·연준)가 강도 높은 긴축을 이어가고 있고 최근 빅스텝(한번에 기준금리 0.50%포인트 인상)을 단행한 한국은행도 계속해서 기준금리를 올릴 것으로 예상돼 시장 금리 상승도 가속할 것으로 예측하고 있다. 금융위원회와 금감원 등 금융 당국은 이런 금리 상승기를 맞아 2금융권에서 대출이 많은 자영업자의 연착륙 유도를 적극적으로 추진할 방침이다. 금융당국은 중·저신용자에 대한 자금 공급을 원활히 하기 위해 2016년부터 중금리 대출 제도를 운용하고 있다. 중금리 대출 공급액은 2016년 1조3천억원에서 지난해 21조5천억원으로 급증했다. \n\n\n\n서울시내 한 시중은행 대출 창구[연합뉴스 자료사진] 2022.7.13 hama@yna.co.kr이 가운데 민간 중금리 대출은 신용 하위 50%인 개인 대출자를 위한 제도로, 업권별 금리 상한 요건을 충족하는 비보증부 신용대출에 대해 인센티브를 부여하는 방식으로 운영하고 있다. 금융위는 최근 민간 중금리 대출의 금리 상한 기준을 합리화하는 내용의 상호금융업·저축은행·여신전문금융업 감독규정 일부 개정 고시의 규정 변경을 예고했다. 이는 최근 금리가 급등하는 가운데 고금리로 대출을 이용하던 중·저신용자가 보다 낮은 중금리대 대출을 이용할 수 있도록 민간 중금리 금리상한 기준을 합리화하기 위한 조치다. 이에 따라 상호금융의 경우 민간 중금리 대출의 금리 상한을 8.5%에서 10.5%로 상향 조정하기로 했다. 여신전문금융업은 신용카드업의 경우 민간 중금리 대출의 금리 상한을 11%에서 13%, 신용카드 외 사업자는 14%에서 15.5%로 설정할 예정이다. 저축은행은 16%에서 17.5%로 올릴 방침이다. 금융당국 관계자는 ""최근 기준금리 인상 등에 따라 신규 신용대출 금리가 크게 올랐다""면서 ""중·저신용자의 기존 대출 금리가 민간 중금리 대출 금리 상한보다 높아지는 경우 금융사는 중·저신용자 대출을 민간 중금리 대출로 취급할 이유가 줄어들게 된다""고 말했다. 그는 ""이렇게 되면 중·저신용자 신용대출 금리가 급등할 가능성이 있다""면서 ""이번 조치는 최근의 금리 상승분을 민간 중금리 대출의 금리 상한 요건에 반영해 금융사에 적절한 인센티브를 부여함으로써 민간 중금리 대출 확대를 유도하기 위한 것""이라고 덧붙였다. president21@yna.co.kr pan@yna.co.kr viva5@yna.co.kr\n]",https://n.news.naver.com/mnews/article/001/0013316949?sid=101,"[건보료만, 원리금, 차주, 중금리대출, 상한선, 고금리, 중저신용자, 이용, 유도, 서울시내, 시중은행, 대출, 창구, 연합뉴스, 자료, 서울연합뉴스, 심재훈, 이지헌, 오주현, 가계, 대출, 평균, 금리, 수준, 경우, 소득, 최저, 생계비, 대출, 원리금, 사람, 파악, 금리, 대부업, 저축은행, 자영업자, 다중, 채무, 급증, 파악, 금융당국, 민간, 중금리대출, 상한선, 재조정, 중저신용자, 보호, 적극적, 방침, 금융권, 금융감독원, 금리, 급등기, 금리, 상승, 가계, 대출, 차주, 상환, 능력, 영향, 분석, 결과, 그래픽, 금리, 상승, 가계대출자, 영향서울연합뉴스, 원형민, 페이스북, LeYN, 트위터, 올해, 현재, 전체, 금융권, 가계, 대출, 평균, 금리, 포인트, 상승, 대출자, 총부채원리금상환비율DSR, 경우, 분석, 기존, 부채, 금액, 증가, 금리인상, 영끌족, 이자부담, 어쩌나서울연합뉴스, 신준희, 서울시내, 시중은행, 대출, 창구, 한국은행, ...]","[최근, 초과, 연합뉴스, 중금리, 심재훈, 이럴, 이날, 원형민, 올해, 오주현]"
1,None,[대출금리 7% 땐 190만명 원리금 못 갚아],"[\n\t\t\t[앵커] 이렇게 다양한 대응 방안이 시도되고 있지만, 금리 인상 자체를 막을 순 없겠죠.지금처럼 금리가 올라서 가계대출의 평균금리가 7%에 도달하면, 190만명에 달하는 사람이 최저 생계비를 빼면 원리금도 못 갚는 것으로 파악됐습니다.엄하은 기자, 정확히 어떤 통계가 나온 건가요?[기자]총부채원리금상환비율인 DSR이 70%를 초과하면, 일반적으로 소득에서 최저 생계비를 뺐을 때 원리금을 감당하지 못하는 차주로 분류됩니다.금융권에 따르면 올해 3월 말 기준 전체 금융권 가계 대출은 1616조 2천억원인데요.3.96%인 평균 금리가 3%p 상승하면 대출자 1646만명 중 DSR이 70%를 초과하는 경우가 190만명에 달하는 것으로 분석됐습니다.기존 140만명에서 50만명이 늘어난 것인데, 이들의 부채 금액은 122조 9천억원 늘어, 480조 4천억원으로 증가하게 됩니다.소득에서 소득세와 건강보험료 등만 차감해도 원리금을 못 갚는 대출자를 의미하는 DSR 90% 초과 차주들도 30만명 증가하는 것으로 파악됐는데요.특히, 금리가 높은 대부업, 저축은행 등 2금융권과 자영업자의 다중 채무가 급증하는 것으로 나타났습니다.[앵커]문제는 앞으로 금리가 더 오른다는 거잖아요?[기자]미국 연방준비제도가 강도 높은 긴축을 이어가고 있고, 여기에 맞춰 한국은행도 기준금리

In [25]:
for idx, series in X_regulation_contents[["6.추진배경및정부개입필요성", "7.규제내용", "9.도입목표및기대효과","10.비용편익분석"]].iteritems():
    best_nouns = list()
    filter = NounsFilter_Tfidf_and_Freq(pd.Series(series))
    best_nouns.append(filter.pick_the_best_K_nouns(K=10).values)
    X_regulation_contents[f"{series.name}_best_nouns"] = best_nouns

X_regulation_contents

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,detail_url,subject,date,cagegory1,cagegory2,cagegory3,fname,download_url,1.규제사무명,2.규제조문,3.위임법령,4.유형,5.입법예고,6.추진배경및정부개입필요성,7.규제내용,8.피규제집단및이해관계자,9.도입목표및기대효과,10.비용편익분석,6.추진배경및정부개입필요성_nouns,7.규제내용_nouns,9.도입목표및기대효과_nouns,10.비용편익분석_nouns,6.추진배경및정부개입필요성_best_nouns,7.규제내용_best_nouns,9.도입목표및기대효과_best_nouns,10.비용편익분석_best_nouns
0,https://www.fsc.go.kr/po040301/view?noticeId=3810&curPage=1&srchKey=&srchText=&srchBeginDt=&srchEndDt=,상호금융업감독규정 일부개정고시(안) 규정변경예고,2022-07-13,구분규정변경예고,법률구분행정규칙,예고기간2022-07-13 ~ 2022-07-24,3.상호금융업감독규정규제영향분석서.hwp,https://www.fsc.go.kr/comm/getFile?srvcId=RULENOTICE&upperNo=3810&fileTy=ATTACH&fileNo=4,민간중금리대출 금리상한 조정,상호금융업 감독규정 제4조의6,신용협동조합법 시행령 제16조의2제1호,신설,2022.07.13 2022.07.24 규제의 필요성,최근 한국은행 기준금리 인상 등에 따라 신규 신용대출 금리가 크게 상승 금리 상승으로 중저신용자 대출금리가 민간 중금리대출 금리 상한보다 높아지는 경우 금융회사가 금리상한에 맞추어 중저신용자 대출을 민간 중금리 대출로 취급할 유인 감소 - 이에 따라 중저신용자 신용대출 금리가 더욱 크게 상승할 가능성 금리상승이 민간중금리 금리상한 요건에 반영되도록 기준을 합리화하여 금융회사에 적절한 인센티브를 부여함으로써 민간 중금리 대출 확대를 유도할 필요,비조합원 대출한도 우대가 적용되는 민간중금리 대출 적격요건 중 금리요건을 기존의 민간중금리 금리상한100분의 8.5을 기준으로 조달금리 변동 폭을 고려하여 금융위원회가 정하는 금리상한으로 변경하며 이 경우에도 금리상한 한도는 100분의 10.5로 설정,상호금융업권,중금리대출 금리요건 합리화를 통해 금융회사에게 적절한 인센티브를 부여함으로써 기존의 민간중금리 대출 금리상한을 벗어나 고금리로 대출을 이용하던 중저신용자가 보다 낮은 중금리대 대출을 이용 규제의 적정성,단위백만원 비용 편익 순비용 피규제자 피규제자 이외 정성분석 조달금리 하락시 금융회사의 인센티브 축소 조달금리를 반영하는 민간중금리 금리요건 합리화로 중금리대출 활성화 주요내용 조달금리 상승시에는 민간중금리 대출 금리상한이 상승함에 따라 금융회사에게는 인센티브가 확대되고 고금리로 대출받던 중저신용자에게는 보다 낮은 금리로 대출을 받을 수 있는 기회가 증가 조달금리 하락시에는 민간중금리 대출 금리상한이 하락함에 따라 금융회사의 인센티브는 축소되나 이는 조달비용 하락에 따른 효과로서 금융회사에게 실질적 부담이 발생한다고 보기 어려운 측면이 있으며. 중저신용자는 낮은 금리로 중금리대출 활용 가능 민간중금리 기준에 조달금리 변경을 반영함으로써 금융회사와 중저신용자에 대한 순편익 증대,"[한국은행, 기준금리, 인상, 신규, 신용대출, 금리, 상승, 금리, 상승, 중저신용자, 대출금리, 민간, 중금리대출, 금리, 상한, 경우, 금융회사, 금리상한, 중저신용자, 대출, 민간, 중금리, 대출, 취급, 감소, 중저신용자, 신용대출, 금리, 상승, 가능성, 금리상승, 민간중금리, 금리상한, 요건, 반영, 기준, 합리, 금융회사, 적절, 인센티브, 민간, 중금리, 대출, 확대, 유도, 필요]","[조합원, 대출한도, 우대, 적용, 민간중금리, 대출, 적격요건, 금리요건, 기존, 민간중금리, 금리상한, 기준, 조달금리, 변동, 금융위원회, 금리상한, 변경, 경우, 금리상한, 설정]","[중금리대출, 금리요건, 합리화, 금융회사, 적절, 인센티브, 기존, 민간중금리, 대출, 금리상한, 고금리, 대출, 이용, 중저신용자, 중금리대, 대출, 이용, 규제, 적정성]","[단위백만원, 비용, 편익, 순비용, 피규제, 피규제, 이외, 정성분석, 조달금리, 하락시, 금융회사, 인센티브, 축소, 조달금리, 반영, 민간중금리, 금리요건, 합리화, 중금리대출, 활성화, 주요내용, 조달금리, 상승시, 민간중금리, 대출, 금리상한, 상승, 금융회사, 인센티브, 확대, 고금리, 대출, 중저신용자에게, 금리, 대출, 기회, 조달금리, 하락시, 민간중금리, 대출, 금리상한, 하락, 금융회사, 인센티브, 축소, 조달비용, 하락, 효과, 금융회사, 실질적, 부담, 발생, 측면, 중저신용자, 금리, 중금리대출, 활용, 가능, 민간중금리, 기준, 조달금리, 변경, 반영함, 금융회사, 중저신용자, 순편익, 증대]","[가능성, 중저신용자, 중금리대출, 중금리, 인상, 신용대출, 신규, 필요, 민간중금리, 기준금리]","[대출, 대출한도, 민간중금리, 변동, 설정, 적격요건, 조달금리, 금리상한]","[금리요건, 민간중금리, 적정성, 중금리대, 중금리대출, 이용, 대출]","[가능, 편익, 증대, 중금리대출, 주요내용, 조달비용, 정성분석, 활성화, 이외, 활용]"


### 3-2. 자연어분석을 위한 데이터셋 
- 규제 데이터: `X_regulation_contents`
- 규제에 대한 뉴스 데이터: `X_news_contents`

In [26]:
X_news_contents, X_regulation_contents #text data pandas

(    date                                       title  \
 0   None  [대출금리 7% 땐 190만명 '최저 생계비 빼면' 원리금 못 갚는다]      
 1   None  [대출금리 7% 땐 190만명 원리금 못 갚아]                   
 2   None  [금리상승에 중금리 상한 합리화..중금리 대출 확대 유도]             
 3   None  [중금리대출 은행 금리 상한 6.79% 상향]                    
 4   None  [사채 내몰리지 않게… 저신용자 중금리대출 숨통 튼다]               
 5   None  ["본인 명의 차량은 있죠?" 무담보 급전 대출 시장서 담보 요구 많아졌다]   
 6   None  [대출금리 7% 땐 190만명 '최저 생계비 빼면' 원리금 못 갚는다]      
 7   None  [대출금리 7% 땐 190만명 원리금 못 갚아]                   
 8   None  [금리상승에 중금리 상한 합리화..중금리 대출 확대 유도]             
 9   None  [은행권 중금리 대출 이자 상한 6.5→6.79%로 조정]             
 10  None  [사채 내몰리지 않게… 저신용자 중금리대출 숨통 튼다]               
 11  None  ["본인 명의 차량은 있죠?" 무담보 급전 대출 시장서 담보 요구 많아졌다]   
 12  None  [대출금리 7% 땐 190만명 원리금 못 갚아]                   
 13  None  [금리상승에 중금리 상한 합리화..중금리 대출 확대 유도]             
 14  None  [은행권 중금리 대출 이자 상한 6.5→6.79%로 조정]             
 15  None  [사채 내몰리지 않게… 저신용자 중금리대출 숨통 튼다]               
 16  None  ["본인 명의 차량은 있죠?" 무담보

## 4. 규제에 대한 **종목 영향도**, **핵심 주제**, **피규제 종목** 분석 

### 4-1. FinBert를 이용한 규제에 대한 종목 영향도 분석
- 규제 정보로 인한 종목 영향도 분석
- 한국어, 한국주식 분석을 위한 방법론
    - https://github.com/ProsusAI/finBERT

In [27]:
## 

### 4-2. KoBERTopic 활용한 규제와 규제에 대한 뉴스들의 핵심 주제 추출
- `BertTopic()` 활용
- http://insightcampus.co.kr:9090/insightcommunity/?mod=document&uid=12969


4-2-1. 뉴스의 핵심 주제 추출 (KoBERTopic)



In [28]:
# BERTopic 모델 사용 - 이유 : LDA, NMF 보다 더 높은 정확성

In [29]:
# 패키지 설치
!pip install bertopic
!pip install bertopic[visualization]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [30]:
# Mecab 설치
!git clone https://github.com/SOMJANG/Mecab-ko-for-Google-Colab.git
%cd Mecab-ko-for-Google-Colab
!bash install_mecab-ko_on_colab190912.sh

fatal: destination path 'Mecab-ko-for-Google-Colab' already exists and is not an empty directory.
/content/gdrive/Shareddrives/Dacon_krx/code/Mecab-ko-for-Google-Colab
Installing konlpy.....
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Done
Installing mecab-0.996-ko-0.9.2.tar.gz.....
from https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
--2022-07-21 13:52:05--  https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
Resolving bitbucket.org (bitbucket.org)... 104.192.141.1, 2406:da00:ff00::22cd:e0db, 2406:da00:ff00::6b17:d1f5, ...
Connecting to bitbucket.org (bitbucket.org)|104.192.141.1|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://bbuseruploads.s3.amazonaws.com/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz?response-content-disposition=attachment%3B%20filename%3D%22mecab-0.996-ko-0.9.2.tar.gz%22&response-content-encoding=None&AWSAccess

In [31]:
# 필요한 함수 불러오기
from tqdm import tqdm # 진행률을 표시하기 위한 패키지
from sklearn.feature_extraction.text import CountVectorizer # Countvectorizer : 단어들의 출현빈도로 여러 문서들을 벡터화한다. 소문자 변환
from konlpy.tag import Mecab
from bertopic import BERTopic

In [32]:
# 데이터 전처리
X_news_contents['clean_content'] = ""

for idx in range(len(X_news_contents['content'])): 
  content = X_news_contents['content'][idx]
  clean_content = TextCleaner.clean_ko_sentences(content)
  X_news_contents['clean_content'].loc[idx] = clean_content


X_news_contents['clean_content']

0     건보료만 내도 원리금 못 갚는 차주 120만명금융당국 2금융권 중금리대출 상한선 올려 고금리 중저신용자 이용 유도 서울시내 한 시중은행 대출 창구 연합뉴스 자료 사진 2022.7.13 hama yna.co.kr 서울연합뉴스 심재훈 이지헌 오주현 기자 가계 대출 평균 금리가 7 수준에 이를 경우 소득에서 최저 생계비를 빼면 대출 원리금도 못 갚는 사람이 190만명에 달하는 것으로 파악됐다. 특히 금리가 높은 대부업 저축은행 등 2금융권과 자영업자의 다중 채무가 급증하는 것으로 파악돼 금융당국은 2금융권의 민간 중금리대출 상한선을 재조정하는 등 중저신용자 보호에 적극적으로 나설 방침이다. 18일 금융권 등에 따르면 금융감독원이 최근 금리 급등기를 맞아 금리 상승이 가계 대출 차주의 상환 능력에 미치는 영향을 분석했더니 이런 결과가 나왔다. 그래픽 금리 3 p 상승 시 가계대출자에 미치는 영향서울연합뉴스 원형민 기자 circlemin yna.co.kr 페이스북 tuney.kr LeYN1 트위터 yonhap graphics 올해 3월 말 현재 전체 금융권의 가계 대출은 1천616조2천억원으로 평균 금리3.96 가 3 포인트 상승하면 대출자 1천646만명 중 총부채원리금상환비율DSR이 70 를 넘는 경우가 190만명에 달하는 것으로 분석됐다. 이는 기존 140만명에서 50만명이 늘어난 것이며 이들의 부채 금액은 357조5천억원에서 480조4천억원으로 122조9천억원이나 증가하게 된다. 금리인상에 영끌족 이자부담 어쩌나서울연합뉴스 신준희 기자 13일 서울시내 한 시중은행 대출 창구. 한국은행 금통위는 이날 기준금리를 한꺼번에 0.50 포인트 올렸다. 이에 따라 대출금리가 기준금리 인상 폭만큼만 올라도 가계대출자의 이자 부담은 24조원 가까이 불어날 것으로 추정된다. 2022.7.13 hama yna.co.kr DSR이란 1년 동안 갚아야 하는 대출이자와 대출 원금이 소득과 비교해 얼마나 되는지를 계산한 수치다. DSR이 70 를 초과하는 경우 일반적으로 소득에

In [33]:
X_news_contents['clean_content'].size

17

In [34]:
# 데이터형 확인 (Series)
type(X_news_contents['clean_content'])

pandas.core.series.Series

In [35]:
# 데이터형 변환 (Series to list)
X_news_clean = X_news_contents['clean_content'].to_list()
X_news_clean

['건보료만 내도 원리금 못 갚는 차주 120만명금융당국 2금융권 중금리대출 상한선 올려 고금리 중저신용자 이용 유도 서울시내 한 시중은행 대출 창구 연합뉴스 자료 사진 2022.7.13 hama yna.co.kr 서울연합뉴스 심재훈 이지헌 오주현 기자 가계 대출 평균 금리가 7 수준에 이를 경우 소득에서 최저 생계비를 빼면 대출 원리금도 못 갚는 사람이 190만명에 달하는 것으로 파악됐다. 특히 금리가 높은 대부업 저축은행 등 2금융권과 자영업자의 다중 채무가 급증하는 것으로 파악돼 금융당국은 2금융권의 민간 중금리대출 상한선을 재조정하는 등 중저신용자 보호에 적극적으로 나설 방침이다. 18일 금융권 등에 따르면 금융감독원이 최근 금리 급등기를 맞아 금리 상승이 가계 대출 차주의 상환 능력에 미치는 영향을 분석했더니 이런 결과가 나왔다. 그래픽 금리 3 p 상승 시 가계대출자에 미치는 영향서울연합뉴스 원형민 기자 circlemin yna.co.kr 페이스북 tuney.kr LeYN1 트위터 yonhap graphics 올해 3월 말 현재 전체 금융권의 가계 대출은 1천616조2천억원으로 평균 금리3.96 가 3 포인트 상승하면 대출자 1천646만명 중 총부채원리금상환비율DSR이 70 를 넘는 경우가 190만명에 달하는 것으로 분석됐다. 이는 기존 140만명에서 50만명이 늘어난 것이며 이들의 부채 금액은 357조5천억원에서 480조4천억원으로 122조9천억원이나 증가하게 된다. 금리인상에 영끌족 이자부담 어쩌나서울연합뉴스 신준희 기자 13일 서울시내 한 시중은행 대출 창구. 한국은행 금통위는 이날 기준금리를 한꺼번에 0.50 포인트 올렸다. 이에 따라 대출금리가 기준금리 인상 폭만큼만 올라도 가계대출자의 이자 부담은 24조원 가까이 불어날 것으로 추정된다. 2022.7.13 hama yna.co.kr DSR이란 1년 동안 갚아야 하는 대출이자와 대출 원금이 소득과 비교해 얼마나 되는지를 계산한 수치다. DSR이 70 를 초과하는 경우 일반적으로 소득에서 최저

In [36]:
# 빈 문자열이거나 숫자로만 이루어진 줄은 제외
preprocessed_documents = []

for line in tqdm(X_news_clean):
  if line and not line.replace(' ', '').isdecimal():# isdemical : 주어진 문자열이 int형으로 변환 가능한지 확인하는 함수
    preprocessed_documents.append(line)

preprocessed_documents

100%|██████████| 17/17 [00:00<00:00, 42594.49it/s]


['건보료만 내도 원리금 못 갚는 차주 120만명금융당국 2금융권 중금리대출 상한선 올려 고금리 중저신용자 이용 유도 서울시내 한 시중은행 대출 창구 연합뉴스 자료 사진 2022.7.13 hama yna.co.kr 서울연합뉴스 심재훈 이지헌 오주현 기자 가계 대출 평균 금리가 7 수준에 이를 경우 소득에서 최저 생계비를 빼면 대출 원리금도 못 갚는 사람이 190만명에 달하는 것으로 파악됐다. 특히 금리가 높은 대부업 저축은행 등 2금융권과 자영업자의 다중 채무가 급증하는 것으로 파악돼 금융당국은 2금융권의 민간 중금리대출 상한선을 재조정하는 등 중저신용자 보호에 적극적으로 나설 방침이다. 18일 금융권 등에 따르면 금융감독원이 최근 금리 급등기를 맞아 금리 상승이 가계 대출 차주의 상환 능력에 미치는 영향을 분석했더니 이런 결과가 나왔다. 그래픽 금리 3 p 상승 시 가계대출자에 미치는 영향서울연합뉴스 원형민 기자 circlemin yna.co.kr 페이스북 tuney.kr LeYN1 트위터 yonhap graphics 올해 3월 말 현재 전체 금융권의 가계 대출은 1천616조2천억원으로 평균 금리3.96 가 3 포인트 상승하면 대출자 1천646만명 중 총부채원리금상환비율DSR이 70 를 넘는 경우가 190만명에 달하는 것으로 분석됐다. 이는 기존 140만명에서 50만명이 늘어난 것이며 이들의 부채 금액은 357조5천억원에서 480조4천억원으로 122조9천억원이나 증가하게 된다. 금리인상에 영끌족 이자부담 어쩌나서울연합뉴스 신준희 기자 13일 서울시내 한 시중은행 대출 창구. 한국은행 금통위는 이날 기준금리를 한꺼번에 0.50 포인트 올렸다. 이에 따라 대출금리가 기준금리 인상 폭만큼만 올라도 가계대출자의 이자 부담은 24조원 가까이 불어날 것으로 추정된다. 2022.7.13 hama yna.co.kr DSR이란 1년 동안 갚아야 하는 대출이자와 대출 원금이 소득과 비교해 얼마나 되는지를 계산한 수치다. DSR이 70 를 초과하는 경우 일반적으로 소득에서 최저

In [37]:
class CustomTokenizer:
    def __init__(self, tagger):
        self.tagger = tagger
    def __call__(self, sent):
        sent = sent[:1000000]
        word_tokens = self.tagger.morphs(sent)
        result = [word for word in word_tokens if len(word) > 1]
        return result

In [38]:
custom_tokenizer = CustomTokenizer(Mecab())

In [39]:
vectorizer = CountVectorizer(tokenizer=custom_tokenizer, max_features=3000)

In [40]:
# 모델 선언
model = BERTopic(embedding_model="sentence-transformers/xlm-r-100langs-bert-base-nli-stsb-mean-tokens", \
                 vectorizer_model=vectorizer,
                 nr_topics="auto",
                 calculate_probabilities=True)

In [41]:
topics, probs = model.fit_transform(preprocessed_documents)

Downloading:   0%|          | 0.00/574 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.06k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/731 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/122 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/527 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/229 [00:00<?, ?B/s]

/usr/local/lib/python3.7/dist-packages/numba/np/ufunc/parallel.py:363: NumbaWarning: The TBB threading layer requires TBB version 2019.5 or later i.e., TBB_INTERFACE_VERSION >= 11005. Found TBB_INTERFACE_VERSION = 9107. The TBB threading layer is disabled.
  warnings.warn(problem)


IndexError: ignored

In [ ]:
# 상대적 빈도로 생성된 토픽
# -1번 토픽은 이상치로 분류된 것으로 제외 처리
model.get_topic_freq().head()

In [ ]:
# 0번 토픽이 할당된 단어
# 연습데이터 결과 괜찮은듯(정치 분야)
model.get_topic(0)

In [ ]:
# TF-IDF 표현(2D) : 상호작용형 Plotly 모형
model.visualize_topics()

In [ ]:
# 확률 시각화
model.visualize_distribution(probs[0])

In [ ]:
# 50번째 토픽까지 키워드 보기
for i in range(0, 50):
  print(i,'번째 토픽 :', model.get_topic(i))

### 4-3. 상장 종목 정보를 활용하여 해당 규제가 어떤 종목에 매칭되는지 분류
- 매칭 분류 분석
- KRX 상장종목 정보 리스트 활용

In [ ]:
## 

## 5. 제공 정보 활용성 분석을 위한 visualization

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

## 끝